In [1]:
import findspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, sum, desc

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

23/07/25 17:47:06 WARN Utils: Your hostname, RNTDELL000700 resolves to a loopback address: 127.0.1.1; using 192.168.1.49 instead (on interface wlp0s20f3)
23/07/25 17:47:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/25 17:47:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/25 17:47:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Los datos adjuntos a esta lección forman parte de la base de datos Football Data from Transfermarkt de Kaggle. El conjunto de datos se compone de varios archivos CSV con información sobre competiciones, juegos, clubes, jugadores y apariciones.

In [2]:
df_appearances = spark.read.option('header', 'true').csv('./data/football/appearances.csv')
df_clubs = spark.read.option('header', 'true').csv('./data/football/clubs.csv')
df_competitions = spark.read.option('header', 'true').csv('./data/football/competitions.csv')
df_games = spark.read.option('header', 'true').csv('./data/football/games.csv')
df_leagues = spark.read.option('header', 'true').csv('./data/football/leagues.csv')
df_players = spark.read.option('header', 'true').csv('./data/football/players.csv')

In [3]:
df_appearances.show(5)

+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|player_id|game_id|appearance_id|competition_id|player_club_id|goals|assists|minutes_played|yellow_cards|red_cards|
+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|    52453|2483937|2483937_52453|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2479929|2479929_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2483937|2483937_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2484582|2484582_67064|           RU1|         28095|    0|      0|            55|           0|        0|
|    67064|2485965|2485965_67064|           RU1|         28095|    0|      0|            90|           0|        0|
+---------+-------+-------------+--------------+--------------+-----+---

In [4]:
df_clubs.show(5)

+-------+-------------------+-------------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------------+--------------------+
|club_id|               name|        pretty_name|domestic_competition_id|total_market_value|squad_size|average_age|foreigners_number|foreigners_percentage|national_team_players|        stadium_name|stadium_seats|net_transfer_record|      coach_name|                 url|
+-------+-------------------+-------------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------------+--------------------+
|   1429|          cesena-fc|          Cesena Fc|                    IT1|              5.09|        30|       24.0|                3|                 10.0|                    1|"Orogel St

In [5]:
df_competitions.show(5)

+--------------+---------------+------------------+----------+------------+--------------------+-------------+--------------------+
|competition_id|           name|              type|country_id|country_name|domestic_league_code|confederation|                 url|
+--------------+---------------+------------------+----------+------------+--------------------+-------------+--------------------+
|            L1|     bundesliga|        first_tier|        40|     Germany|                  L1|       europa|https://www.trans...|
|           DFB|      dfb-pokal|      domestic_cup|        40|     Germany|                  L1|       europa|https://www.trans...|
|           DFL|   dfl-supercup|domestic_super_cup|        40|     Germany|                  L1|       europa|https://www.trans...|
|           NL1|     eredivisie|        first_tier|       122| Netherlands|                 NL1|       europa|https://www.trans...|
|           NLP|toto-knvb-beker|      domestic_cup|       122| Netherlands| 

In [6]:
df_games.show(5)

+-------+----------------+------+-------------+----------+------------+------------+---------------+---------------+------------------+------------------+--------------------+----------+-----------------+--------------------+
|game_id|competition_code|season|        round|      date|home_club_id|away_club_id|home_club_goals|away_club_goals|home_club_position|away_club_position|             stadium|attendance|          referee|                 url|
+-------+----------------+------+-------------+----------+------------+------------+---------------+---------------+------------------+------------------+--------------------+----------+-----------------+--------------------+
|2457642|            NLSC|  2014|        Final|2014-08-03|        1269|         610|              1|              0|              null|              null| Johan Cruijff ArenA|     42000|   Danny Makkelie|https://www.trans...|
|2639088|            BESC|  2013|        Final|2014-07-20|          58|         498|            

In [118]:
df_leagues.show(5)

+---------+------------------+-------------+
|league_id|              name|confederation|
+---------+------------------+-------------+
|       L1|        bundesliga|       europa|
|      NL1|        eredivisie|       europa|
|      BE1|jupiler-pro-league|       europa|
|      ES1|            laliga|       europa|
|      PO1|          liga-nos|       europa|
+---------+------------------+-------------+
only showing top 5 rows



In [8]:
df_players.show(5)

+---------+---------------+-------------------+-------------------+----------------+----------------------+-------------+----------+--------------+-----+------------+-------------------+---------------------------+--------------------+
|player_id|current_club_id|               name|        pretty_name|country_of_birth|country_of_citizenship|date_of_birth|  position|  sub_position| foot|height_in_cm|market_value_in_gbp|highest_market_value_in_gbp|                 url|
+---------+---------------+-------------------+-------------------+----------------+----------------------+-------------+----------+--------------+-----+------------+-------------------+---------------------------+--------------------+
|    38790|          28095|     dmitri-golubov|     Dmitri Golubov|           UdSSR|                Russia|   1985-06-24|    Attack|Centre-Forward| Both|         178|               null|                   675000.0|https://www.trans...|
|   106539|          28095| aleksandr-vasiljev| Aleksand

Determine los tres países con mayor número de jugadores (jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.

In [12]:
(df_players.groupBy('country_of_birth')
    .count()
    .orderBy(desc('count'))
).na.drop().show(3)

+----------------+-----+
|country_of_birth|count|
+----------------+-----+
|          France| 1694|
|           Spain| 1388|
|           Italy| 1312|
+----------------+-----+
only showing top 3 rows



Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

In [114]:
df_rojas = df_appearances.groupBy('player_id').agg(sum('red_cards').alias('total_red_cards')).orderBy(desc('total_red_cards'))

In [115]:
df_rojas.show(5)

+---------+---------------+
|player_id|total_red_cards|
+---------+---------------+
|   131821|            5.0|
|   177843|            5.0|
|    45146|            5.0|
|   111455|            5.0|
|     4811|            5.0|
+---------+---------------+
only showing top 5 rows



In [89]:
df_nombre = df_players.select(['player_id', 'pretty_name'])

In [91]:
df_nombre.show(5)

+---------+-------------------+
|player_id|        pretty_name|
+---------+-------------------+
|    38790|     Dmitri Golubov|
|   106539| Aleksandr Vasiljev|
|   164389|      Rory Donnelly|
|    45247|     Damien Perquis|
|    97205|Nikita Bezlikhotnov|
+---------+-------------------+
only showing top 5 rows



In [110]:
df_nombre_rojas = df_nombre.join(df_rojas, 'player_id').drop('player_id').orderBy(desc('total_red_cards'))

In [116]:
df_nombre_rojas.show(truncate=False)

+----------------------+---------------+
|pretty_name           |total_red_cards|
+----------------------+---------------+
|Thomas Lam            |5.0            |
|Domenico Berardi      |5.0            |
|Mario Balotelli       |5.0            |
|Granit Xhaka          |5.0            |
|Hilton                |5.0            |
|Ivan Ordets           |4.0            |
|Kalidou Koulibaly     |4.0            |
|Rodrigo De Paul       |4.0            |
|Rodrigo               |4.0            |
|David Luiz            |4.0            |
|Jordi Amat            |4.0            |
|Rafik Halliche        |4.0            |
|Yannick Cahuzac       |4.0            |
|Roberto Soriano       |4.0            |
|Jordan Amavi          |4.0            |
|Damien Da Silva       |4.0            |
|Ognjen Vranjes        |4.0            |
|Timothee Kolodziejczak|4.0            |
|Fallou Diagne         |4.0            |
|Allan Nyom            |4.0            |
+----------------------+---------------+
only showing top

¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.

In [145]:
premier_code = df_leagues.where(col('name') == 'premier-league').drop('confederation')
premier_code.show()

+---------+--------------+
|league_id|          name|
+---------+--------------+
|      GB1|premier-league|
+---------+--------------+



In [144]:
premier_games = df_games.filter(col('competition_code') == 'GB1').dropDuplicates(['game_id']).groupBy('competition_code').count()
premier_games.show()

+----------------+-----+
|competition_code|count|
+----------------+-----+
|             GB1| 2809|
+----------------+-----+



In [151]:
total_games = premier_code.join(premier_games, col('league_id') == col('competition_code')).drop(col('league_id'), col('competition_code'))
total_games.show()

+--------------+-----+
|          name|count|
+--------------+-----+
|premier-league| 2809|
+--------------+-----+



Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

In [157]:
total_attendance = df_games.groupBy('competition_code').agg(sum('attendance').alias('total_attendance'))

In [161]:
league_attendance = df_leagues.join(total_attendance, col('league_id') == col('competition_code')).orderBy(desc('total_attendance')).drop(col('league_id'), col('confederation'), col('competition_code'))
league_attendance.show()

+--------------------+----------------+
|                name|total_attendance|
+--------------------+----------------+
|      premier-league|     8.6964852E7|
|          bundesliga|     7.8102473E7|
|              laliga|     6.2943533E7|
|             serie-a|     5.3475147E7|
|             ligue-1|     5.1593963E7|
|          eredivisie|     3.4572418E7|
|        premier-liga|     2.0878744E7|
|            liga-nos|     2.0072843E7|
|  jupiler-pro-league|     1.7817099E7|
|           super-lig|     1.7455236E7|
|scottish-premiership|     1.7379753E7|
|         superligaen|       7945555.0|
|      super-league-1|       6417136.0|
|        premier-liga|       4944837.0|
+--------------------+----------------+

